In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from time import sleep

In [2]:
# Mở trình duyệt Google Chrome
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito') # Tab ẩn danh
options.add_argument("--start-maximized") # Full window
# options.add_argument('headless') # Không hiển thị chrome
driver = webdriver.Chrome(executable_path='../chromedriver.exe', options=options)

C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_7836\1376250944.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='../chromedriver.exe', options=options)


In [3]:
# Tìm url sp theo keyword
keyword = 'Tai nghe Bluetooh'

url = 'https://tiki.vn/search?q=%s'%('%20'.join(keyword.split(' ')))
driver.get(url)
sleep(1)

element= WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="CatalogProducts__Wrapper-sc-1r8ct7c-0 fKlIZg"]')))
html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
soup = BeautifulSoup(html_of_interest, 'lxml')

raw_links = soup.select('a[data-view-id="product_list_item"]')
list_link = []

for num_link in range(len(raw_links)):
    link = raw_links[num_link].attrs['href']

    if 'tka.tiki.vn' in link:
        list_link.append('https:' + link)
    else:
        list_link.append('https://tiki.vn' + link)

In [105]:
# Kéo từ từ xuống dưới cùng trang mới hiện thông tin, mô tả và đánh giá
total_height = int(driver.execute_script("return document.body.scrollHeight"))

for i in range(1, total_height, 5):
    driver.execute_script("window.scrollTo(0, {});".format(i))

check_total_height = int(driver.execute_script("return document.body.scrollHeight"))
if check_total_height > total_height:
    for i in range(total_height, check_total_height, 5):
        driver.execute_script("window.scrollTo(0, {});".format(i))

In [124]:
# Test
link_product = 'https://tiki.vn/apple-iphone-15-p271966786.html?spid=271966788'

# Crawl thông tin sp theo url
driver.get(link_product)
sleep(1)

WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'main > div[class="ContainerRevamp-sc-1hvvgwz-0 dOGdaN"]')))

total_height = int(driver.execute_script("return document.body.scrollHeight"))

for i in range(1, total_height, 5):
    driver.execute_script("window.scrollTo(0, {});".format(i))

check_total_height = int(driver.execute_script("return document.body.scrollHeight"))
if check_total_height > total_height:
    for i in range(total_height, check_total_height, 5):
        driver.execute_script("window.scrollTo(0, {});".format(i))

element= WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'main > div[class="ContainerRevamp-sc-1hvvgwz-0 dOGdaN"]')))
html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
soup = BeautifulSoup(html_of_interest, 'lxml')

In [125]:
# Tên sản phẩm
soup.select('h1[class="Title__TitledStyled-sc-1kxsq5b-0 cvyKhs"]')[0].text

'Apple iPhone 15'

In [126]:
# Thương hiệu
soup.select('a[data-view-id="pdp_details_view_brand"]')[0].text

'Apple'

In [127]:
# Số lượng bán
soup.select('div[data-view-id="pdp_quantity_sold"]')[0].text.split()[-1]

'302'

In [128]:
# Số lượt đánh giá
soup.select('a[data-view-id="pdp_main_view_review"]')[0].text.strip('()')

'42'

In [129]:
# Đánh giá sao
soup.select('div[class="styles__StyledReview-sc-1swui9f-1 dXPbue"] > div')[0].text

'4.8'

In [130]:
# Giá bán
soup.select('div[class="product-price__current-price"]')[0].text.replace('₫', '')

'21.490.000'

In [131]:
# Thông số lựa chọn (Màu sắc, dung lượng,...)
list_option = soup.select('div[class="styles__ProductOptionsWrapper-sc-18rzur4-0 jZCObm"] > div')
for option in list_option:
    print()

In [132]:
# Loại option
option_label = soup.select('div[class="styles__ProductOptionsWrapper-sc-18rzur4-0 jZCObm"] > div')[0].attrs['data-view-label']


In [133]:
# Tên option
soup.select('div[class="styles__ProductOptionsWrapper-sc-18rzur4-0 jZCObm"] > div')[0].select('div[class="styles__OptionListWrapper-sc-1pikfxx-2 jgDdBJ"] > div')[3].text

'Xanh Lá'

In [134]:
# Tên shop
soup.select('span[class="seller-name"]')[0].text

'Tiki TradingTheo DõiChat'

In [135]:
# Đánh giá sao shop
soup.select('div[class="item review"] > div')[0].text

'4.7'

In [136]:
# Số lượt đánh giá shop
soup.select('div[class="item review"] > div')[1].text.strip('()đánh giá')

'5.4tr+'

In [137]:
# Thông tin sản phẩm
sleep(1)
product_info_html = driver.find_elements(By.CSS_SELECTOR, 'div[class="WidgetTitle__WidgetContainerStyled-sc-1ikmn8z-0 iHMNqO"]')[-5].get_attribute('innerHTML')
product_info_soup = BeautifulSoup(product_info_html, 'lxml')
product_info = product_info_soup.select('div > div > div > span')

for i in range(0, len(product_info), 2):
    info_label = product_info[i].text
    info_value = product_info[i+1].text
    print(info_label + ': ' + info_value)

Dung lượng pin: 3349 mAh
Bluetooth: v5.3
Thương hiệu: Apple
Xuất xứ thương hiệu: Mỹ
Hỗ trợ thẻ nhớ ngoài: Không
Chip đồ họa (GPU): Apple GPU 5 nhân
Chip set: Apple A16 Bionic 6 nhân
Chức năng khác: Bảo mật nâng cao: Mở khoá khuôn mặt Face IDTính năng đặc biệt: Phát hiện va chạm (Crash Detection)Âm thanh Dolby AtmosKháng nước, bụi: IP68SOS Khẩn Cấp
Kết nối khác: NFC
Tốc độ CPU: 3.46 GHz
Đèn Flash: Có
Kích thước: Dài 147.6 mm - Ngang 71.6 mm - Dày 7.8 mm
Loại/ Công nghệ màn hình: OLED
Ghi âm: Ghi âm mặc định
GPS: BEIDOU, QZSS, GPS, GLONASS, GALILEO
Hỗ trợ 5G: Có
Phụ kiện đi kèm: Cáp, Sách hướng dẫn, Cây lấy sim
Model: iPhone 15
Jack tai nghe: Type-C
Số sim: 1 khe sim vật lý
Loại pin: Li-Ion
Loại Sim: 1 SIM vật lý và 1 E-SIM
Chất liệu: Khung nhôm & Mặt lưng kính cường lực
Nghe nhạc: AAC, MP3, Apple Lossless, FLAC, Dolby Digital, Dolby Digital Plus và Dolby Atmos
Xuất xứ (Made in): China
Cổng sạc: Type-C
Trọng lượng sản phẩm: 171 g
Quay phim: HD 720p@30fps, FullHD 1080p@60fps, FullHD 1080p

In [138]:
# Mô tả sản phẩm
product_describe_html = driver.find_elements(By.CSS_SELECTOR, 'div[class="WidgetTitle__WidgetContainerStyled-sc-1ikmn8z-0 iHMNqO"]')[-4].get_attribute('innerHTML')
product_describe = BeautifulSoup(product_describe_html, 'lxml').text
product_describe

'Mô tả sản phẩmNội dung quảng cáo\niPhone 15 sở hữu Dynamic Island, camera Chính 48MP và USB-C trong thiết kế bằng kính pha màu và nhôm bền bỉ.\n\xa0\nTính năng nổi bật\n\nDYNAMIC ISLAND RA MẮT TRÊN IPHONE 15 — Dynamic Island hiển thị linh động các cảnh báo và Hoạt Động Trực Tiếp, nhờ đó bạn sẽ không bỏ lỡ thông tin khi đang làm việc khác. Bạn có thể xem ai đang gọi đến, kiểm tra tình hình chuyến bay của bạn và hơn thế nữa.\nTHIẾT KẾ SÁNG TẠO — iPhone 15 sở hữu thiết kế bằng kính pha màu và nhôm bền bỉ. Và có khả năng chống tia nước, chống nước và chống bụi.1 Mặt trước Ceramic Shield bền chắc hơn mọi mặt kính điện thoại thông minh. Và màn hình Super Retina XDR 6,1"2 sáng gấp đôi dưới ánh mặt trời so với iPhone 14.\nCAMERA CHÍNH 48MP VỚI TELEPHOTO 2X — Camera Chính 48MP chụp ảnh với độ phân giải siêu cao. Vậy nên bạn có thể chụp được những bức ảnh nổi bật có độ chi tiết ấn tượng dễ dàng hơn bao giờ hết. Camera Telephoto với độ thu phóng quang học 2x giúp bạn bố cục ảnh chụp cận cảnh hoà

In [139]:
# Đánh giá
driver.find_element(By.CSS_SELECTOR, 'div[class="customer-reviews__inner"]').location_once_scrolled_into_view

{'x': 270, 'y': 0}